In [ ]:
# import all the modules
from nipype.pipeline.engine import Workflow, Node, MapNode
from nipype.interfaces.utility import IdentityInterface, Function
from nipype.interfaces.io import SelectFiles, DataSink, FreeSurferSource
from nipype.interfaces.fsl.preprocess import FAST
from nipype.interfaces.fsl.utils import Reorient2Std
from nipype.interfaces.freesurfer import FSCommand, MRIConvert, Binarize
from nipype.interfaces.ants.registration import RegistrationSynQuick
from pandas import DataFrame, Series, read_csv

# FSL setup- change default file output type
from nipype.interfaces.fsl import FSLCommand
FSLCommand.set_default_output_type('NIFTI_GZ')

#freesurfer setup
fs_dir = '/moochie/Cat/Aggregate_anats/subjects_dir'
FSCommand.set_default_subjects_dir(fs_dir)

# Study specific variables
analysis_home = '/home/camachocm2/Analysis/aggregate_anats'

workflow_dir = analysis_home + '/workflows'
proc_dir = analysis_home + '/proc/subj_data'
group_dir = analysis_home + '/proc/group_data'

#subject_info = read_csv(analysis_home + '/misc/subjs.csv', index_col=0)
#all_subjects = subject_info['fsID'].tolist()
template_subjects_list = analysis_home + '/misc/template_subs.txt'
template_subjects = open(template_subjects_list).read().splitlines()
template_subjects = [str(template_subjects[0])]

#set up datasink
substitutions = [('_subject_id_','')]
datasink = Node(DataSink(substitutions=substitutions,
                         base_directory = proc_dir),
                name = 'datasink')

## 1. Template Brain Creation
This section creates a template representative of the sample based on a predetermined list.

In [ ]:
##### Set up nodes for file handling #####

template_source = MapNode(FreeSurferSource(subjects_dir = fs_dir), 
                          name = 'template_source', 
                          iterfield = ['subject_id'])
template_source.inputs.subject_id = template_subjects

In [ ]:
######### Template creation functions #########
def make3DTemplate(subject_T1s, num_proc, output_prefix):
    from nipype import config, logging
    config.enable_debug_mode()
    logging.update_logging(config)
    
    from os.path import abspath, split
    from os import getcwd
    from shutil import copyfile
    from glob import glob
    from subprocess import call

    curr_dir = getcwd()

    #copy T1s into current directory
    for T in range(0,len(subject_T1s)):
        [dirname,filename] = split(subject_T1s[T])
        copyfile(subject_T1s[T],curr_dir + '/S' + str(T)+'_'+filename)

    # -c flag is control for local computing (2= use localhost; required for -j flag)
    # -j flag is for number of processors allowed
    call(['antsMultivariateTemplateConstruction2.sh', '–d','3','–o', output_prefix,'–r','1','–c','2','–j', str(num_proc), '*.nii.gz'])
    
    sample_template = abspath(output_prefix + 'template0.nii.gz')
    
    return(sample_template)


In [ ]:
######### Template creation nodes #########

#convert freesurfer brainmask files to .nii
convertT1 = Node(MRIConvert(out_file='T1.nii.gz',
                            out_type='niigz'), 
                 name='convertT1')

#reorient files to standard space
reorientT1 = MapNode(Reorient2Std(),
                     name = 'reorientT1',
                     iterfield = ['in_file'])

#pass files into template function (normalized, pre-skull-stripping)
makeTemplate = Node(Function(input_names=['subject_T1s','num_proc','output_prefix'],
                             output_names=['sample_template'],
                             function=make3DTemplate),
                    name='makeTemplate')
makeTemplate.inputs.num_proc=4 
makeTemplate.inputs.output_prefix='Child_'

In [ ]:
######### Template creation workflow #########
template_flow = Workflow(name = "template_flow")
template_flow.connect([(template_source, convertT1, [('T1','in_file')]),
                       (convertT1, reorientT1, [('out_file', 'in_file')]),
                       (reorientT1, makeTemplate, [('out_file', 'subject_T1s')]),
                       (reorientT1, datasink, [('out_file','proc_T1s')]),
                       (makeTemplate, datasink, [('sample_template', 'sample_template')])
                      ])

template_flow.base_dir = workflow_dir
template_flow.write_graph(graph2use = 'flat')
template_flow.run(plugin='MultiProc',plugin_args={'memory_gb':10})

## 2. Tissue Segmentation
This next section creates the 4 tissue segmentations plus a brainmask for cortical thickness estimation tissue priors:
1. CSF 
2. cortical gray matter 
3. white matter 
4. subcortical gray matter  
5. whole brain

In [ ]:
## data handling nodes

# subject handling node
infosource = Node(IdentityInterface(fields=['subject_id']), 
                  name='infosource')
infosource.iterables = [('subject_id', template_subjects)]


#pull freesurfer data
tissue_source = Node(FreeSurferSource(subjects_dir = fs_dir),
                     name = 'tissue_source')

In [ ]:
## Tissue labeling functions

def relabel_fast(fast_tissue_list):
    from nipype import config, logging
    from os.path import split
    from os import rename
    config.enable_debug_mode()
    logging.update_logging(config)
    tissue_list = sorted(fast_tissue_list)
    csf = tissue_list[0]
    wm = tissue_list[2]
    [wd, csf_file] = split(csf)
    [wd, wm_file] = split(wm)
    rename(csf, wd + 'csf.nii.gz')
    rename(wm, wd + 'wm.nii.gz')
    wm_csf = [wd + 'csf.nii.gz', wd + 'wm.nii.gz']
    return(wm_csf)

# Create subcortical and cortical gray matter masks <-- custom function. inputs: fs aseg + tissue class files
def aseg_to_tissuemaps(aseg):
    from nipype import config, logging
    config.enable_debug_mode()
    logging.update_logging(config)
    from nibabel import load, save, Nifti1Image
    from numpy import zeros_like
    from os.path import abspath
    aseg_nifti = load(aseg)
    aseg_data = aseg_nifti.get_data()
    cortical_labels = [3, 42]
    subcortical_labels =[10, 11, 12, 13, 17, 18, 26, 49, 50, 51, 52, 53, 54, 58, 28, 60]
    brainstem_labels = [16]
    cerebellum_labels = [6, 7, 8, 45, 46, 47]

    #creating array of zeroes that replaces 0's with 1's when matches values of subcortical_labels
    cortical_data = zeros_like(aseg_data)
    for x in cortical_labels:
        cortical_data[aseg_data == x] = 1
    cortical_nifti = Nifti1Image(cortical_data, aseg_nifti.affine)
    
    subcort_data = zeros_like(aseg_data) 
    for x in subcortical_labels:
        subcort_data[aseg_data == x] = 1
    subcort_nifti = Nifti1Image(subcort_data, aseg_nifti.affine)
    
    bs_data = zeros_like(aseg_data) 
    for x in brainstem_labels:
        bs_data[aseg_data == x] = 1
    bs_nifti = Nifti1Image(bs_data, aseg_nifti.affine)
    
    cb_data = zeros_like(aseg_data) 
    for x in cerebellum_labels:
        cb_data[aseg_data == x] = 1
    cb_nifti = Nifti1Image(cb_data, aseg_nifti.affine)
    
    save(subcort_nifti, 'subcortical_gm.nii.gz')
    save(cortical_nifti, 'cortical_gm.nii.gz')
    save(bs_nifti, 'brainstem.nii.gz')
    save(cb_nifti, 'cerebellum.nii.gz')
    subcort_file = abspath('subcortical_gm.nii.gz')
    cortical_file = abspath('cortical_gm.nii.gz')
    brainstem_file = abspath('brainstem.nii.gz')
    cerebellum_file = abspath('cerebellum.nii.gz')
    gm_list = [subcort_file, cortical_file, brainstem_file, cerebellum_file]
    return(gm_list)

def combine_seg(file_list_1, file_list_2):
    from nipype import config, logging
    config.enable_debug_mode()
    logging.update_logging(config)
    from nibabel import load, save, Nifti1Image
    from numpy import zeros_like
    from os.path import abspath

    in_files = file_list_1 + file_list_2

    for tissue in in_files:
        if 'segmentcsf' in tissue:
            csf_file = tissue
            csf_nifti = load(csf_file)
            csf_data = csf_nifti.get_data()
        elif 'subcortical' in tissue:
            subcortical_file = tissue
            sc_nifti = load(subcortical_file)
            sc_data = sc_nifti.get_data()
        elif 'cortical' in tissue:
            cortex_file = tissue
            cort_nifti = load(cortex_file)
            cort_data = cort_nifti.get_data()
        elif 'segmentwm' in tissue:
            wm_file = tissue
            wm_nifti = load(wm_file)
            wm_data = wm_nifti.get_data()
        elif 'brainstem' in tissue:
            brainstem_file = tissue
            bs_nifti = load(brainstem_file)
            bs_data = bs_nifti.get_data()
        elif 'cerebellum' in tissue:
            cerebellum_file = tissue
            cb_nifti = load(cerebellum_file)
            cb_data = cb_nifti.get_data()

    combined_labels_data = zeros_like(csf_data)

    # these are done in a deliberate order, later tissues overwrite previous ones
    combined_labels_data[csf_data>0] = 1
    combined_labels_data[wm_data>0] = 4
    combined_labels_data[bs_data>0] = 5
    combined_labels_data[sc_data>0] = 3
    combined_labels_data[cb_data>0] = 6
    combined_labels_data[cort_data>0] = 2

    combined_labels_nii = Nifti1Image(combined_labels_data, csf_nifti.affine)
    combined_labels_file = save(combined_labels_nii, 'combined_tissues.nii.gz')
    tissue_map = abspath('combined_tissues.nii.gz')
    
    return(tissue_map)

In [ ]:
## tissue labeling nodes

#convert freesurfer brainmask files to .nii
convert_to_nii = Node(MRIConvert(out_file='brain.nii.gz',
                                 out_type='niigz'), 
                      name='convert_to_nii', 
                      iterfield = ['in_file'])

#reorient brainmask file to standard
reorient_to_std = Node(Reorient2Std(),
                       name = 'reorient_to_std',
                       iterfield = ['in_file'])

# Reorient aseg to standard
reorient_aseg = Node(Reorient2Std(),
                     name = 'reorient_aseg',
                     iterfield = ['in_file'])

#T1 gets run through segmentation (2) ---> results in segmentation into 3 tissue classes (wm, gm, csf)
segment = Node(FAST(number_classes = 3, 
                    segments=True, 
                    no_bias=True), 
               name = 'segment', 
               iterfield = ['in_files'])

# Convert freesurfer aseg to nii
convert_aseg = Node(MRIConvert(out_file='aseg.nii.gz',
                               out_type='niigz'), 
                    name='convert_aseg',
                    iterfield = ['in_file'])

# Split aseg into to types of gray matter
aseg_to_gm = Node(Function(input_names=['aseg'],
                           output_names=['gm_list'],
                           function=aseg_to_tissuemaps),
                  name='aseg_to_gm', 
                  iterfield=['aseg'])

# Relabel the FAST segmentation 
relabel_fast_seg = Node(Function(input_names=['fast_tissue_list'],
                                 output_names=['wm_csf'],
                                 function=relabel_fast),
                        name='relabel_fast_seg',
                        iterfield=['fast_tissue_list'])

#combine tissue segmentation to 1 volume for easier editing of tissue priors
combine_labels = Node(Function(input_names=['file_list_1', 'file_list_2'], 
                               output_names=['tissue_map'], 
                               function=combine_seg),
                      name='combine_labels')

In [ ]:
######### Tissue segmentation workflow #########
segment_flow = Workflow(name = 'segment_flow')
segment_flow.connect([(infosource, tissue_source, [('subject_id','subject_id')]),
                      (tissue_source, convert_to_nii, [('brainmask','in_file')]),
                      (convert_to_nii, reorient_to_std, [('out_file', 'in_file')]),
                      (reorient_to_std, segment, [('out_file', 'in_files')]),
                      (segment, relabel_fast_seg, [('tissue_class_files', 'fast_tissue_list')]),
                      (tissue_source, convert_aseg, [('aseg','in_file')]),
                      (convert_aseg, reorient_aseg, [('out_file', 'in_file')]),
                      (reorient_aseg, aseg_to_gm, [('out_file', 'aseg')]),
                      (aseg_to_gm, combine_labels, [('gm_list','file_list_1')]),
                      (relabel_fast_seg, combine_labels, [('fast_tissue_list','file_list_2')]),
                      
                      (combine_labels, datasink, [('tissue_map','combined_labels')])
                     ])

segment_flow.base_dir = workflow_dir
segment_flow.write_graph(graph2use = 'flat')
segment_flow.run('MultiProc', plugin_args={'n_procs': 1, 'memory_gb':4})

## 3. Tissue Priors
This section creates the tissue priors (after manual editing the ouputs from the previous step)

In [ ]:
### Data handling nodes
template= proc_dir + '/template/lcbd_template0.nii.gz'
file_template = {'brainmask': proc_dir + '/template_brainmask/{subject_id}/brainmask.nii.gz',
                 'sc_gm': proc_dir + '/template_gm/{subject_id}/subcortical_gm.nii.gz',
                 'c_gm': proc_dir + '/template_gm/{subject_id}/cortical_gm.nii.gz',
                 'csf': proc_dir + '/template_wm_csf/{subject_id}/segmentcsf.nii.gz',
                 'wm': proc_dir + '/template_wm_csf/{subject_id}/segmentwm.nii.gz', 
                 'bs': proc_dir + '/template_gm/{subject_id}/brainstem.nii.gz', 
                 'cb': proc_dir + '/template_gm/{subject_id}/cerebellum.nii.gz'}
template_files = Node(SelectFiles(file_template), name='template_files')

In [ ]:
### Tissue priors nodes

# register to template
reg_to_template = Node(RegistrationSynQuick(fixed_image=template), 
                       name = 'reg_to_template')

# apply the transform to the tissue maps


# average the tissue maps



In [ ]:
tissuepriors_flow = Workflow(name = 'tissuepriors_flow')
tissuepriors_flow.connect([(infosource, tissue_source, [('subject_id','subject_id')]),
                           (tissue_source, convert_to_nii, [('T1','in_file')]),
                           (convert_to_nii, reorient_to_std, [('out_file','in_file')]),
                           (reorient_to_std, reg_to_template, [('')])
                          ])

tissuepriors_flow.base_dir = workflow_dir
tissuepriors_flow.write_graph(graph2use = 'flat')
tissuepriors_flow.run('MultiProc', plugin_args={'n_procs': 4, 'memory_gb':10})